In [1]:
import cvlib
from pytorchvideo.transforms import (
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Resize,
)
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pytorchvideo.data.encoded_video import EncodedVideo
import pandas as pd


/home/prsood/projects/def-whkchun/prsood/sarcasm_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/IEMOCAP_df.pkl")
df

,text,Session,timings,audio_path,video_path,emotion_label,"V,A,D",emotion,speaker,sentiment,sentiment_label,split
0,What's he going to say?,Ses01F_script01_1_F000,"(6.21, 9.32)",../../data/iemocap_wav/Ses01F_script01_1_F000.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,frustrated,"[2.0000, 2.3333, 2.3333]",0,True,0,frustrated,train
1,He saw it.,Ses01F_script01_1_M000,"(11.9892, 15.1219)",../../data/iemocap_wav/Ses01F_script01_1_M000.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 1.6667, 2.3333]",1,False,1,neutral,train
2,He was out here when it broke.,Ses01F_script01_1_M001,"(19.0456, 22.3962)",../../data/iemocap_wav/Ses01F_script01_1_M001.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.3333]",1,False,1,neutral,train
3,Four o clock this morning. I heard a crack a...,Ses01F_script01_1_M002,"(24.8, 35.9971)",../../data/iemocap_wav/Ses01F_script01_1_M002.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[2.6667, 3.0000, 2.3333]",1,False,1,neutral,train
4,I don't know.,Ses01F_script01_1_M003,"(38.66, 40.53)",../../data/iemocap_wav/Ses01F_script01_1_M003.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.0000]",1,False,1,neutral,train
...,...,...,...,...,...,...,...,...,...,...,...,...
7375,"[LAUGHTER] That was quit a rouser, wasn't it?",Ses05M_script03_1_M030,"(215.4, 220.75)",../../data/iemocap_wav/Ses05M_script03_1_M030.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 3.5000]",4,True,4,excited,val
7376,"Oh, I didn't hit you very hard.",Ses05M_script03_1_M031,"(221.13, 223.83)",../../data/iemocap_wav/Ses05M_script03_1_M031.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[2.5000, 3.5000, 3.5000]",4,True,4,excited,val
7377,I should never forget his face. [LAUGHTER],Ses05M_script03_1_M032,"(229.88, 233.06)",../../data/iemocap_wav/Ses05M_script03_1_M032.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.5000, 4.5000, 4.5000]",4,True,4,excited,val
7378,We were very much younger then.,Ses05M_script03_1_M033,"(234.14, 236.66)",../../data/iemocap_wav/Ses05M_script03_1_M033.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,excited,"[4.0000, 3.0000, 3.5000]",4,True,4,excited,val


In [3]:
df[df['split'] == "train"]

,text,Session,timings,audio_path,video_path,emotion_label,"V,A,D",emotion,speaker,sentiment,sentiment_label,split
0,What's he going to say?,Ses01F_script01_1_F000,"(6.21, 9.32)",../../data/iemocap_wav/Ses01F_script01_1_F000.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,frustrated,"[2.0000, 2.3333, 2.3333]",0,True,0,frustrated,train
1,He saw it.,Ses01F_script01_1_M000,"(11.9892, 15.1219)",../../data/iemocap_wav/Ses01F_script01_1_M000.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 1.6667, 2.3333]",1,False,1,neutral,train
2,He was out here when it broke.,Ses01F_script01_1_M001,"(19.0456, 22.3962)",../../data/iemocap_wav/Ses01F_script01_1_M001.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.3333]",1,False,1,neutral,train
3,Four o clock this morning. I heard a crack a...,Ses01F_script01_1_M002,"(24.8, 35.9971)",../../data/iemocap_wav/Ses01F_script01_1_M002.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[2.6667, 3.0000, 2.3333]",1,False,1,neutral,train
4,I don't know.,Ses01F_script01_1_M003,"(38.66, 40.53)",../../data/iemocap_wav/Ses01F_script01_1_M003.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,neutral,"[3.0000, 2.0000, 2.0000]",1,False,1,neutral,train
...,...,...,...,...,...,...,...,...,...,...,...,...
4428,"I know that, mom. Just, you help me stay here.",Ses05M_script01_1_M038,"(335.88, 340.05)",../../data/iemocap_wav/Ses05M_script01_1_M038.wav,../../data/iemocap_avi/Ses05M_script01_1.avi,frustrated,"[2.0000, 4.5000, 4.5000]",0,True,0,frustrated,train
4429,Just don't think that way. Do you hear me?,Ses05M_script01_1_F036,"(339.57, 342.03)",../../data/iemocap_wav/Ses05M_script01_1_F036.wav,../../data/iemocap_avi/Ses05M_script01_1.avi,angry,"[2.0000, 2.5000, 3.5000]",2,False,2,angry,train
4430,"I don't understand you, do I?",Ses05M_script01_1_F037,"(346.27, 348.79)",../../data/iemocap_wav/Ses05M_script01_1_F037.wav,../../data/iemocap_avi/Ses05M_script01_1.avi,frustrated,"[2.0000, 2.5000, 2.5000]",0,False,0,frustrated,train
4431,"No, you don't. I'm a pretty tough guy.",Ses05M_script01_1_M040,"(348.43, 353.6)",../../data/iemocap_wav/Ses05M_script01_1_M040.wav,../../data/iemocap_avi/Ses05M_script01_1.avi,frustrated,"[1.5000, 4.0000, 4.5000]",0,True,0,frustrated,train


In [2]:
df = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
0,2,0,0,why do all you re coffee mugs have numbers on ...,13,test,../../data/test_splits_wav/dia0_utt0.wav,dia0_utt0,surprise,6,36181,217086,../../data/test_video/dia0_utt0.mp4,positive,2,"(0.08076116071428571, 2.0190290178571426)"
1,3,0,1,oh that s so monica can keep track that way if...,28,test,../../data/test_splits_wav/dia0_utt1.wav,dia0_utt1,anger,6,108203,649218,../../data/test_video/dia0_utt1.mp4,negative,0,"(0.30100982937685455, 5.638917470326409)"
2,0,0,2,y'know what ?,3,test,../../data/test_splits_wav/dia0_utt2.wav,dia0_utt2,neutral,6,18773,112638,../../data/test_video/dia0_utt2.mp4,neutral,1,"(0.525967672413793, 1.0114762931034482)"
3,0,1,0,"come on , lydia , you can do it",9,test,../../data/test_splits_wav/dia1_utt0.wav,dia1_utt0,neutral,6,32085,192510,../../data/test_video/dia1_utt0.mp4,neutral,1,"(0.020053125, 1.7646750000000002)"
4,1,1,1,push !,2,test,../../data/test_splits_wav/dia1_utt1.wav,dia1_utt1,joy,6,10923,65538,../../data/test_video/dia1_utt1.mp4,positive,2,"(0.3516875, 0.4344375)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13500,4,113,9,no,1,val,../../data/val_splits_wav/dia113_utt9.wav,dia113_utt9,sadness,6,17408,104448,../../data/val_video/dia113_utt9.mp4,negative,0,"(0.20148148148148148, 0.38281481481481483)"
13501,4,113,10,what ? oh my god ! i m gonna miss you so much !,14,val,../../data/val_splits_wav/dia113_utt10.wav,dia113_utt10,sadness,6,44032,264192,../../data/val_video/dia113_utt10.mp4,negative,0,"(0.04017518248175182, 2.5712116788321167)"
13502,4,113,11,i m gonna miss you !,6,val,../../data/val_splits_wav/dia113_utt11.wav,dia113_utt11,sadness,6,28672,172032,../../data/val_video/dia113_utt11.mp4,negative,0,"(0.020134831460674157, 0.6644494382022471)"
13503,4,113,12,i mean it s the end of an era !,10,val,../../data/val_splits_wav/dia113_utt12.wav,dia113_utt12,sadness,6,29355,176130,../../data/val_video/dia113_utt12.mp4,negative,0,"(0.14112980769230768, 1.7338804945054944)"


In [11]:
# df = ...
for i in range(3,14):
    sub_df = df.iloc[i*1000:(i+1)*1000]
    sub_df['bbox'] = sub_df.progress_apply(lambda x: boxes(x['video_path'] , x['timings'] ) if x['split'] == "train" else None, axis = 1 )
    sub_df.to_pickle(f"{args.dataset}_sub_{i}.pkl")

      emotion  dialog  utterance  \
3000        0      41          4   
3001        2      41          5   
3002        0      41          6   
3003        0      41          7   
3004        0      41          8   
...       ...     ...        ...   
3995        1     155          9   
3996        0     155         10   
3997        0     155         11   
3998        0     155         12   
3999        4     156          0   

                                                   text  num_words  split  \
3000  remember when you were a little kid , i used t...         24  train   
3001                                     since then ? !          4  train   
3002                     no , it 's only been six years          8  train   
3003  i just wanted to put a nice memory in your hea...         24  train   
3004                                                joe          1  train   
...                                                 ...        ...    ...   
3995                        

In [8]:
df_val = df[df['split'] == "val"]
df_val_dia4 = df_val[df_val['dialog'] == 4]
df_val_dia4

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
12434,0,4,0,yes ? yes ? ! how can i help you ?,11,val,../../data/val_splits_wav/dia4_utt0.wav,dia4_utt0,neutral,6,44032,264192,../../data/val_video/dia4_utt0.mp4,neutral,1,"(0.0, 2.691737226277372)"
12435,0,4,1,"yeah , we were we were just looking around",9,val,../../data/val_splits_wav/dia4_utt1.wav,dia4_utt1,neutral,6,29013,174078,../../data/val_video/dia4_utt1.mp4,neutral,1,"(0.0, 1.2894666666666668)"
12436,2,4,2,"oh oh , you re you re fellow scholars",9,val,../../data/val_splits_wav/dia4_utt2.wav,dia4_utt2,surprise,6,38229,229374,../../data/val_video/dia4_utt2.mp4,negative,0,"(0.3614086134453781, 2.18852993697479)"
12437,0,4,3,"what exactly were you looking for , hmm ?",9,val,../../data/val_splits_wav/dia4_utt3.wav,dia4_utt3,neutral,6,40277,241662,../../data/val_video/dia4_utt3.mp4,neutral,1,"(0.24166200000000002, 2.41662)"
12438,0,4,4,"perhaps , perhaps dr chester stock s musings o...",13,val,../../data/val_splits_wav/dia4_utt4.wav,dia4_utt4,neutral,6,115029,690174,../../data/val_video/dia4_utt4.mp4,neutral,1,"(0.32041504178272984, 5.987756093314763)"
12439,0,4,5,uhh,1,val,../../data/val_splits_wav/dia4_utt5.wav,dia4_utt5,neutral,6,18773,112638,../../data/val_video/dia4_utt5.mp4,neutral,1,"(0.20229525862068964, 0.46527909482758617)"
12440,0,4,7,"uh , actually i find marion s views far too pr...",11,val,../../data/val_splits_wav/dia4_utt7.wav,dia4_utt7,neutral,6,73728,442368,../../data/val_video/dia4_utt7.mp4,neutral,1,"(0.0, 4.047026086956523)"
12441,0,4,8,i find marion s views far too progressionist,8,val,../../data/val_splits_wav/dia4_utt8.wav,dia4_utt8,neutral,6,52907,317442,../../data/val_video/dia4_utt8.mp4,neutral,1,"(0.2404863636363636, 2.5050662878787877)"
12442,2,4,9,"i m sorry , who are you ?",8,val,../../data/val_splits_wav/dia4_utt9.wav,dia4_utt9,surprise,6,50859,305154,../../data/val_video/dia4_utt9.mp4,positive,2,"(0.0603548259493671, 2.7159671677215194)"
12443,0,4,10,"i m a professor here uh , ross geller",9,val,../../data/val_splits_wav/dia4_utt10.wav,dia4_utt10,neutral,6,35499,212994,../../data/val_video/dia4_utt10.mp4,neutral,1,"(0.040339772727272724, 1.9968187499999999)"


In [7]:
tqdm.pandas()
df['new'] = df.progress_apply(lambda x: x['dialog'] + x['utterance'] , axis = 1)

100%|██████████| 13505/13505 [00:00<00:00, 31162.52it/s]


In [9]:
df['new12'] = df.progress_apply(lambda x: [x['dialog'] + x['utterance']] if x['split'] == "train" else [] , axis = 1)

100%|██████████| 13505/13505 [00:00<00:00, 40911.99it/s]


In [12]:
path = "../../data/train_video/dia9_utt1.mp4"

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,new,new12
65,5,9,0,coming through ! oh ! coming through ! oh ! he...,21,test,../../data/test_splits_wav/dia9_utt0.wav,dia9_utt0,fear,6,92843,557058,../../data/test_video/dia9_utt0.mp4,negative,0,"(0.12047102076124569, 5.72237348615917)",9,[]
66,0,9,1,"oh well , it s not so bad",8,test,../../data/test_splits_wav/dia9_utt1.wav,dia9_utt1,neutral,6,29013,174078,../../data/test_video/dia9_utt1.mp4,neutral,1,"(0.5036979166666666, 1.6521291666666666)",10,[]
67,4,9,2,"yeah , most of the damage is pretty mostly con...",13,test,../../data/test_splits_wav/dia9_utt2.wav,dia9_utt2,sadness,6,34133,204798,../../data/test_video/dia9_utt2.mp4,negative,0,"(0.0, 1.9924333726415095)",11,[]
68,2,9,3,oh !,2,test,../../data/test_splits_wav/dia9_utt3.wav,dia9_utt3,surprise,6,16043,96258,../../data/test_video/dia9_utt3.mp4,negative,0,"(0.14324107142857145, 0.1841670918367347)",12,[]
69,2,9,4,my god !,3,test,../../data/test_splits_wav/dia9_utt4.wav,dia9_utt4,surprise,6,4096,24576,../../data/test_video/dia9_utt4.mp4,negative,0,"(0.021333333333333333, 0.256)",13,[]
70,0,9,5,"hey buddy , do you think i can borrow your uni...",14,test,../../data/test_splits_wav/dia9_utt5.wav,dia9_utt5,neutral,6,51541,309246,../../data/test_video/dia9_utt5.mp4,neutral,1,"(0.0, 2.275051953125)",14,[]
71,2,9,6,excuse me ?,3,test,../../data/test_splits_wav/dia9_utt6.wav,dia9_utt6,surprise,6,46763,280578,../../data/test_video/dia9_utt6.mp4,positive,2,"(0.3023469827586207, 0.7054762931034483)",15,[]
72,3,9,7,joey ! he s working ! you would look,9,test,../../data/test_splits_wav/dia9_utt7.wav,dia9_utt7,anger,6,89429,536574,../../data/test_video/dia9_utt7.mp4,negative,0,"(0.921535394265233, 3.9465754928315415)",16,[]
73,4,9,8,"oh , how bad is it ?",7,test,../../data/test_splits_wav/dia9_utt8.wav,dia9_utt8,sadness,6,54955,329730,../../data/test_video/dia9_utt8.mp4,negative,0,"(0.46197551169590645, 1.4461842105263158)",17,[]
74,4,9,9,"oh , it s bad it s really bad the only thing i...",24,test,../../data/test_splits_wav/dia9_utt9.wav,dia9_utt9,sadness,6,147797,886782,../../data/test_video/dia9_utt9.mp4,negative,0,"(1.242328362255965, 6.652468004338394)",18,[]


In [2]:
df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data123_val.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,bbox
0,2,0,0,why do all you re coffee mugs have numbers on ...,13,test,../../data/test_splits_wav/dia0_utt0.wav,dia0_utt0,surprise,6,36181,217086,../../data/test_video/dia0_utt0.mp4,positive,2,"(0.08076116071428571, 2.0190290178571426)",None
1,3,0,1,oh that s so monica can keep track that way if...,28,test,../../data/test_splits_wav/dia0_utt1.wav,dia0_utt1,anger,6,108203,649218,../../data/test_video/dia0_utt1.mp4,negative,0,"(0.30100982937685455, 5.638917470326409)",None
2,0,0,2,y'know what ?,3,test,../../data/test_splits_wav/dia0_utt2.wav,dia0_utt2,neutral,6,18773,112638,../../data/test_video/dia0_utt2.mp4,neutral,1,"(0.525967672413793, 1.0114762931034482)",None
3,0,1,0,"come on , lydia , you can do it",9,test,../../data/test_splits_wav/dia1_utt0.wav,dia1_utt0,neutral,6,32085,192510,../../data/test_video/dia1_utt0.mp4,neutral,1,"(0.020053125, 1.7646750000000002)",None
4,1,1,1,push !,2,test,../../data/test_splits_wav/dia1_utt1.wav,dia1_utt1,joy,6,10923,65538,../../data/test_video/dia1_utt1.mp4,positive,2,"(0.3516875, 0.4344375)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13500,4,113,9,no,1,val,../../data/val_splits_wav/dia113_utt9.wav,dia113_utt9,sadness,6,17408,104448,../../data/val_video/dia113_utt9.mp4,negative,0,"(0.20148148148148148, 0.38281481481481483)",[]
13501,4,113,10,what ? oh my god ! i m gonna miss you so much !,14,val,../../data/val_splits_wav/dia113_utt10.wav,dia113_utt10,sadness,6,44032,264192,../../data/val_video/dia113_utt10.mp4,negative,0,"(0.04017518248175182, 2.5712116788321167)",[]
13502,4,113,11,i m gonna miss you !,6,val,../../data/val_splits_wav/dia113_utt11.wav,dia113_utt11,sadness,6,28672,172032,../../data/val_video/dia113_utt11.mp4,negative,0,"(0.020134831460674157, 0.6644494382022471)",[]
13503,4,113,12,i mean it s the end of an era !,10,val,../../data/val_splits_wav/dia113_utt12.wav,dia113_utt12,sadness,6,29355,176130,../../data/val_video/dia113_utt12.mp4,negative,0,"(0.14112980769230768, 1.7338804945054944)",[]


In [ ]:
df_val = df[df['split'] == "val"]

In [3]:

df_val_dia4 = df_val[df_val['dialog'] == 4]
df_val_dia4

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,bbox
12434,0,4,0,yes ? yes ? ! how can i help you ?,11,val,../../data/val_splits_wav/dia4_utt0.wav,dia4_utt0,neutral,6,44032,264192,../../data/val_video/dia4_utt0.mp4,neutral,1,"(0.0, 2.691737226277372)",[]
12435,0,4,1,"yeah , we were we were just looking around",9,val,../../data/val_splits_wav/dia4_utt1.wav,dia4_utt1,neutral,6,29013,174078,../../data/val_video/dia4_utt1.mp4,neutral,1,"(0.0, 1.2894666666666668)",[]
12436,2,4,2,"oh oh , you re you re fellow scholars",9,val,../../data/val_splits_wav/dia4_utt2.wav,dia4_utt2,surprise,6,38229,229374,../../data/val_video/dia4_utt2.mp4,negative,0,"(0.3614086134453781, 2.18852993697479)",[]
12437,0,4,3,"what exactly were you looking for , hmm ?",9,val,../../data/val_splits_wav/dia4_utt3.wav,dia4_utt3,neutral,6,40277,241662,../../data/val_video/dia4_utt3.mp4,neutral,1,"(0.24166200000000002, 2.41662)",[]
12438,0,4,4,"perhaps , perhaps dr chester stock s musings o...",13,val,../../data/val_splits_wav/dia4_utt4.wav,dia4_utt4,neutral,6,115029,690174,../../data/val_video/dia4_utt4.mp4,neutral,1,"(0.32041504178272984, 5.987756093314763)",[]
12439,0,4,5,uhh,1,val,../../data/val_splits_wav/dia4_utt5.wav,dia4_utt5,neutral,6,18773,112638,../../data/val_video/dia4_utt5.mp4,neutral,1,"(0.20229525862068964, 0.46527909482758617)",[]
12440,0,4,7,"uh , actually i find marion s views far too pr...",11,val,../../data/val_splits_wav/dia4_utt7.wav,dia4_utt7,neutral,6,73728,442368,../../data/val_video/dia4_utt7.mp4,neutral,1,"(0.0, 4.047026086956523)",[]
12441,0,4,8,i find marion s views far too progressionist,8,val,../../data/val_splits_wav/dia4_utt8.wav,dia4_utt8,neutral,6,52907,317442,../../data/val_video/dia4_utt8.mp4,neutral,1,"(0.2404863636363636, 2.5050662878787877)",[]
12442,2,4,9,"i m sorry , who are you ?",8,val,../../data/val_splits_wav/dia4_utt9.wav,dia4_utt9,surprise,6,50859,305154,../../data/val_video/dia4_utt9.mp4,positive,2,"(0.0603548259493671, 2.7159671677215194)",[]
12443,0,4,10,"i m a professor here uh , ross geller",9,val,../../data/val_splits_wav/dia4_utt10.wav,dia4_utt10,neutral,6,35499,212994,../../data/val_video/dia4_utt10.mp4,neutral,1,"(0.040339772727272724, 1.9968187499999999)",[]


ERROR! Session/line number was not unique in database. History logging moved to new session 247


In [2]:
import cv2

In [3]:
print(cv2.getBuildInformation() , "\n \n")



General configuration for OpenCV 4.7.0 =====================================
  Version control:               4.7.0

  Extra modules:
    Location (extra):            /home/prsood/projects/def-whkchun/prsood/opencv-python/opencv_contrib/modules
    Version control (extra):     4.7.0

  Platform:
    Timestamp:                   2023-05-06T10:04:39Z
    Host:                        Linux 3.10.0-1160.80.1.el7.x86_64 x86_64
    CMake:                       3.21.3
    CMake generator:             Ninja
    CMake build tool:            /cvmfs/soft.computecanada.ca/gentoo/2020/usr/bin/ninja
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      F

In [16]:
df0 = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data_sub_0.pkl")
df1 = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data_sub_1.pkl")
df2 = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data_sub_2.pkl")

In [17]:
len(df0) , len(df1) , len(df2)

(1000, 1000, 1000)

In [18]:
pd.concat([df0,df1,df2])

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings,bbox
0,2,0,0,why do all you re coffee mugs have numbers on ...,13,test,../../data/test_splits_wav/dia0_utt0.wav,dia0_utt0,surprise,6,36181,217086,../../data/test_video/dia0_utt0.mp4,positive,2,"(0.08076116071428571, 2.0190290178571426)",None
1,3,0,1,oh that s so monica can keep track that way if...,28,test,../../data/test_splits_wav/dia0_utt1.wav,dia0_utt1,anger,6,108203,649218,../../data/test_video/dia0_utt1.mp4,negative,0,"(0.30100982937685455, 5.638917470326409)",None
2,0,0,2,y'know what ?,3,test,../../data/test_splits_wav/dia0_utt2.wav,dia0_utt2,neutral,6,18773,112638,../../data/test_video/dia0_utt2.mp4,neutral,1,"(0.525967672413793, 1.0114762931034482)",None
3,0,1,0,"come on , lydia , you can do it",9,test,../../data/test_splits_wav/dia1_utt0.wav,dia1_utt0,neutral,6,32085,192510,../../data/test_video/dia1_utt0.mp4,neutral,1,"(0.020053125, 1.7646750000000002)",None
4,1,1,1,push !,2,test,../../data/test_splits_wav/dia1_utt1.wav,dia1_utt1,joy,6,10923,65538,../../data/test_video/dia1_utt1.mp4,positive,2,"(0.3516875, 0.4344375)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,4,40,13,i don t think this number s right !,9,train,../../data/train_splits_wav/dia40_utt13.wav,dia40_utt13,sadness,2,25600,51200,../../data/train_video/dia40_utt13.mp4,negative,0,"(0.28354430379746837, 1.4987341772151899)","[[[121, 4, 210, 226], [28, 54, 118, 223]], [[1..."
2996,4,40,14,oh come on ! !,5,train,../../data/train_splits_wav/dia40_utt14.wav,dia40_utt14,sadness,2,35499,70998,../../data/train_video/dia40_utt14.mp4,negative,0,"(0.18152897727272724, 1.4320619318181815)","[[[59, 31, 118, 163], [127, 43, 189, 221]], [[..."
2997,4,41,0,"gotta go i miss you too , i love you , but it ...",18,train,../../data/train_splits_wav/dia41_utt0.wav,dia41_utt0,sadness,6,126293,757758,../../data/train_video/dia41_utt0.mp4,negative,0,"(0.0, 2.4441221446700507)","[[[11, 0, 68, 221], [57, 40, 209, 223]], [[2, ..."
2998,0,41,2,her name 's ronni she 's a pet mortician,9,train,../../data/train_splits_wav/dia41_utt2.wav,dia41_utt2,neutral,6,67584,405504,../../data/train_video/dia41_utt2.mp4,neutral,1,"(0.3017142857142857, 3.902171428571428)","[[[63, 38, 99, 172], [114, 21, 177, 225]], [[6..."
